In [5]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

from flask import Flask, jsonify, request
from flask_cors import CORS, cross_origin

import pickle
import pandas as pd
import numpy as np
import tensorflow as tf

In [12]:
data = pickle.load( open( "katana-assistant-data.pkl", "rb" ) )
words = data['words']
classes = data['classes']

In [13]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [14]:
p = bow("Load bood pessure for patient", words)
print (p)
print (classes)

found in bag: load
found in bag: for
found in bag: paty
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0]
['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']


In [15]:
# Use pickle to load in the pre-trained model
global graph
graph = tf.get_default_graph()

with open(f'katana-assistant-model.pkl', 'rb') as f:
    model = pickle.load(f)

In [16]:
def classify_local(sentence):
    ERROR_THRESHOLD = 0.25
    
    # generate probabilities from the model
    input_data = pd.DataFrame([bow(sentence, words)], dtype=float, index=['input'])
    results = model.predict([input_data])[0]
    # filter out predictions below a threshold, and provide intent index
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], str(r[1])))
    # return tuple of intent and probability
    
    return return_list

In [22]:
classify_local('Hello, good day!')

found in bag: hello
found in bag: ,
found in bag: good
found in bag: day


[('greeting', '1.0')]

In [30]:
classify_local('How you can assist me?')

found in bag: how
found in bag: you
found in bag: can
found in bag: me


[('options', '0.999988')]

In [31]:
classify_local('Get me to adverse medicine form')

found in bag: me
found in bag: to
found in bag: advers


[('adverse_drug', '0.999995')]

In [32]:
classify_local('Place to log blood pressure')

found in bag: to
found in bag: log
found in bag: blood
found in bag: press


[('blood_pressure', '1.0')]

In [33]:
classify_local('Fetch blood result for patient')

found in bag: blood
found in bag: result
found in bag: for
found in bag: paty


[('blood_pressure_search', '1.0')]

In [34]:
classify_local('Blood pressure monitoring in hospital')

found in bag: blood
found in bag: press
found in bag: hospit


[('blood_pressure', '0.847985')]

In [36]:
classify_local('Look for hospital to monitor blood pressure')

found in bag: look
found in bag: for
found in bag: hospit
found in bag: to
found in bag: blood
found in bag: press


[('hospital_search', '0.967533')]

In [ ]:
app = Flask(__name__)
CORS(app)

@app.route("/katana-ml/api/v1.0/assistant", methods=['POST'])
def classify():
    ERROR_THRESHOLD = 0.25
    
    sentence = request.json['sentence']
    
    # generate probabilities from the model
    input_data = pd.DataFrame([bow(sentence, words)], dtype=float, index=['input'])
    results = model.predict([input_data])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    # return tuple of intent and probability
    
    response = jsonify(return_list)
    return response

# running REST interface, port=5000 for direct test, port=5001 for deployment from PM2
if __name__ == "__main__":
    app.run(debug=False, host='0.0.0.0', port=5001)